# Data Uploader

This notebook demonstrates the process of uploading Pyologger-prepared data in NetCDFs to DiveDB's Iceberg storage.

## Starting your environment (Docker or Local)
DiveDB can be run either within a Docker container or directly on your local machine. Both approaches rely on the same .env configuration file for environment variables, but differ in how file paths are handled. Docker will more closely match a cloud-hosted environment, but local runs are quicker and easier to set up.

###Option 1: Run Jupyter server with Docker

To launch the server using Docker, open the Docker Desktop app and run the following command at the root of the project:
```bash
$ make up
```
This command will launch the Jupyter server using the environment variables defined in your .env file.

#### Understanding expected file paths (Docker):

For local uploads in Docker, DiveDB expects the following variables to be set in your .env file:
- `LOCAL_DATA_PATH`: The path to the local data directory (source of data to be uploaded).
- `LOCAL_ICEBERG_PATH`: The path to the local iceberg directory (destination of data to be uploaded).
- `CONTAINER_DATA_PATH`: The path that the container will see for the data directory.
- `CONTAINER_ICEBERG_PATH`: The path that the container will see for the iceberg directory.

The “LOCAL_” paths refer to folders on your machine. The “CONTAINER_” paths define how those folders are mounted inside the container. We recommend keeping the “CONTAINER_” paths consistent with the values provided in .env.example.

#### Uploads to S3 (Docker):

If uploading to S3, the following variables must also be defined:
- `S3_ENDPOINT`: The endpoint of the S3 bucket.
- `S3_ACCESS_KEY`: The access key of the S3 bucket.
- `S3_SECRET_KEY`: The secret key of the S3 bucket.
- `S3_BUCKET`: Just the name of the S3 bucket.

These values are used regardless of whether you run DiveDB in Docker or locally.

When is the server ready?

You’ll know the server is ready when you see logs like the following:
```
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] Serving notebooks from local directory: /app
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] Jupyter Server 2.14.2 is running at:
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] http://e29d05e13fd0:8888/jupyter/tree
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp]     http://127.0.0.1:8888/jupyter/tree
```

#### Connecting to the Jupyter Kernel from within VSCode

The following steps assume that you're working within VSCode, with both the Python and Jupyter extensions installed. For other workflows, you'll need to follow your workflow-specific steps to connect to the Jupyter Kernel launched by `make up`.

To connect to the Jupyter server in your notebook, follow these steps:
1. Click the "Select Kernel" button at the top right of the page.
1. Pick the "Select another kernel" option in the dropdown menu.
1. Pick the "Existing Jupyter Server" option in the dropdown menu.
1. Now we need to connect to the Jupyter server.
    - If you previously connected to the Jupyter server
        - Pick the "localhost" option in the dropdown menu (or whatever you named it prior)
    - If you have not connected to the Jupyter server before
        - Pick the "Enter the URL of the running Jupyter server" option in the dropdown menu.
        - Enter http://localhost:8888/jupyter
        - Give it a name you'll remember (like "Local DiveDB Jupyter Server")
1. Press the "Reload" icon in the top right of the dropdown menu to see the latest kernel.
1. Pick the "Python 3" option in the dropdown menu.

This will ensure you execute the Jupyter notebook in the correct environment.

### Option 2: Run Jupyter server locally (no Docker)

You can also run the server directly on your machine, without Docker.

The environment variables from .env will still be used—consider loading them into your shell manually (e.g., using dotenv or exporting them yourself), or structure your notebook to load them directly with python-dotenv.

#### Understanding expected file paths (Local):

If running locally (outside Docker), the following variables are still expected:
- `LOCAL_DATA_PATH`: The path to the local data directory.
- `LOCAL_ICEBERG_PATH`: The path to the local iceberg directory.

Unlike the Docker setup, the container-specific variables (CONTAINER_DATA_PATH, CONTAINER_ICEBERG_PATH) are not used. Any paths referenced in your code should point directly to the LOCAL_ paths.

Uploads to S3 (Local):

S3 configuration remains unchanged. The same variables must be set:
- `S3_ENDPOINT`: The endpoint of the S3 bucket.
- `S3_ACCESS_KEY`: The access key of the S3 bucket.
- `S3_SECRET_KEY`: The secret key of the S3 bucket.
- `S3_BUCKET`: The name of the S3 bucket.

⸻

Choose the option that best fits your need. Docker provides a consistent, containerized environment, while the local setup is more flexible for quick iteration and debugging.


## Preparing to upload data:
There are two aspects to any data upload:
1. A netCDF file containing measurements and time
2. The metadata for the measurements
    - This describes the context of the measurements using the following fields:
        - dataset
        - animal
        - deployment
        - logger

There are several ways to define your metadata. 

#### Supplied Metadata Dictionary
If you know the metadata for your measurements, you can pass a dictionary to the `upload_netcdf` function. The dictionary should represent metadata existing in the Metadata database and contain the following fields:
- animal: The animal ID
- deployment: The deployment name
- recording: The recording name


### Uploading netCDF files
The `netcdf_file_path` list contains the paths to the netCDF files that we want to upload. It can point to files on your local machine or on a remote server.
In this example, the file is located in the ../data/files/ directory and is named deployment_data.nc.

The upload_netcdf function will perform the following: 
- use the provided metadata dictionary to extract the metadata for your measurements
- upload the measurements to Delta Lake

The process takes between 20 secs per gigabyte (*note: we can speed this up by parellizing the upload process*).

### Example netCDF File
An example netCDF file can be downloaded here: [https://figshare.com/ndownloader/files/50061330](https://figshare.com/ndownloader/files/50061330) that meets the above requirements and can be used as a template for your own data.

Once you've downloaded that file into the local `DiveDB/files/` subdirectory, you'll either need to rename it to `example_data.nc` or set `example_data_path` in the following examples to the name of the downloaded file. 

### Example 1: Uploading a netCDF file

In [1]:
from dotenv import load_dotenv

load_dotenv()

import os
import importlib
import xarray as xr

from DiveDB.services.duck_pond import DuckPond
from DiveDB.services.notion_orm import NotionORMManager

import DiveDB.services.data_uploader
importlib.reload(DiveDB.services.data_uploader)
from DiveDB.services.data_uploader import DataUploader

notion_manager = NotionORMManager(
    token=os.getenv("NOTION_TOKEN"),
    db_map={
        "Deployment DB": os.getenv("NOTION_DEPLOYMENT_DB"),
        "Recording DB": os.getenv("NOTION_RECORDING_DB"),
        "Logger DB": os.getenv("NOTION_LOGGER_DB"),
        "Animal DB": os.getenv("NOTION_ANIMAL_DB"),
        "Dataset DB": os.getenv("NOTION_DATASET_DB"),
        "Sensor DB": os.getenv("NOTION_SENSOR_DB"),
        "Standardized Channel DB": os.getenv("NOTION_STANDARDIZEDCHANNEL_DB"),
    }
)

# Create DuckPond instance (new Iceberg-based data lake)
duck_pond = DuckPond(os.environ["CONTAINER_ICEBERG_PATH"], notion_manager=notion_manager)
data_uploader = DataUploader(duck_pond=duck_pond)

# See above [Example netCDF File](#example-netcdf-file) for saving an example file 
# to this path; if the file has not been renamed to `example_data.nc`, update the 
# path this variable points to.

# dataset_id = "pale-adult-lion_vid-accel_africa_TW"
# deployments = [
#     "2024-10-21_pale-001", # _Satara-35760-M
# ]

dataset_id = "caca-chmy_juv-adult_CW-PP-KS"
deployments = [
    "2015-09-07_caca-002",
]

# dataset_id = "mile-adult-sese_vdr_argentina_RD-KM"
# deployments = [
#     #"2012-11-02_mile-001",  # SEAL 1 - worst seal with regards to data quality (speed didn't work)
#     #"2012-11-02_mile-002",  # SEAL 2
#     #"2012-11-01_mile-003",  # SEAL 3
#     "2013-11-10_mile-006",  # SEAL 6
#     "2013-11-09_mile-008",  # SEAL 8
#     "2015-11-05_mile-011",
#     "2015-11-05_mile-013",
#     "2015-11-06_mile-014",
# ]

for deployment_id in deployments:
    example_data_path = f"../files/{deployment_id}_output.nc"
    print(f"Dataset: {dataset_id} | Deployment: {deployment_id}")

    ds = xr.open_dataset(example_data_path)

    # Extract required IDs from dataset attributes
    dataset_id = ds.attrs['dataset_info_Dataset_ID']
    animal_id = ds.attrs['animal_info_Animal_ID']
    deployment_id = f"{ds.attrs['deployment_info_Deployment Date']}_{animal_id}"

    print(f"Uploading data for Deployment ID: {deployment_id}, Dataset ID: {dataset_id}, Animal ID: {animal_id}")

    # Cycle through logger IDs found in dataset attributes and upload a recording per logger
    logger_ids = set()
    for key, value in ds.attrs.items():
        if value is None:
            continue
        if "logger_id" in key.lower():
            if isinstance(value, (list, tuple)):
                for v in value:
                    if v:
                        logger_ids.add(str(v).strip())
            else:
                for token in str(value).replace(";", ",").split(","):
                    token = token.strip()
                    if token:
                        logger_ids.add(token)

    if not logger_ids:
        print("No logger IDs found; uploading a single recording without logger-specific metadata.")
        metadata = {
            "dataset": dataset_id,
            "animal": animal_id,
            "deployment": deployment_id,
            "recording": f"{deployment_id}_{animal_id}"
        }
        data_uploader.upload_netcdf(example_data_path, metadata)
    else:
        for logger_id in sorted(logger_ids):
            metadata = {
                "dataset": dataset_id,
                "animal": animal_id,
                "deployment": deployment_id,
                "recording": f"{deployment_id}_{animal_id}_{logger_id}"
            }
            data_uploader.upload_netcdf(example_data_path, metadata)


Dataset: caca-chmy_juv-adult_CW-PP-KS | Deployment: 2015-09-07_caca-002
Uploading data for Deployment ID: 2015-09-07_caca-002, Dataset ID: caca-chmy_juv-adult_CW-PP-KS, Animal ID: caca-002
No logger IDs found; uploading a single recording without logger-specific metadata.
Processing 9 coordinate(s) with 11 total variable(s) in the netCDF file.


Processing variables: 100%|██████████| 11/11 [01:02<00:00,  5.72s/it]


Upload Performance Summary
Step                           Time (s)        % of Total     
------------------------------------------------------------
file_loading                         0.32            0.5%
validation                           0.00            0.0%
renaming                             0.00            0.0%
event_processing                     2.17            3.3%
time_coord_precompute                0.09            0.1%
variable_processing                 62.92           95.9%
view_refresh                         0.01            0.0%
------------------------------------------------------------
TOTAL                               65.58          100.0%

Upload complete.


In [28]:
ds

<xarray.Dataset> Size: 558MB
Dimensions:                       (dives_samples: 2249241, dives_variables: 3,
                                   clock_samples: 2249241,
                                   stroke_rate_samples: 2249241,
                                   light_samples: 2249241,
                                   velocity_samples: 2249241,
                                   prh_samples: 2249241, prh_variables: 4,
                                   ...
                                   position_variables: 4,
                                   logger_status_samples: 2249241,
                                   feeding_attempts_samples: 2249241,
                                   location_samples: 2249241,
                                   location_variables: 2,
                                   depth_samples: 2249241, event_data_samples: 2)
Coordinates: (12/16)
  * dives_samples                 (dives_samples) datetime64[ns] 18MB 2015-11...
  * dives_variables               (dives_variables) <U29 348B 'dive_num' ... ...
  * clock_samples                 (clock_samples) datetime64[ns] 18MB 2015-11...
  * stroke_rate_samples           (stroke_rate_samples) datetime64[ns] 18MB 2...
  * light_samples                 (light_samples) datetime64[ns] 18MB 2015-11...
  * velocity_samples              (velocity_samples) datetime64[ns] 18MB 2015...
    ...                            ...
  * logger_status_samples         (logger_status_samples) datetime64[ns] 18MB ...
  * feeding_attempts_samples      (feeding_attempts_samples) datetime64[ns] 18MB ...
  * location_samples              (location_samples) datetime64[ns] 18MB 2015...
  * location_variables            (location_variables) <U9 72B 'latitude' 'lo...
  * depth_samples                 (depth_samples) datetime64[ns] 18MB 2015-11...
  * event_data_samples            (event_data_samples) datetime64[ns] 16B 201...
Data variables: (12/17)
    signal_data_dives             (dives_samples, dives_variables) float64 54MB ...
    signal_data_clock             (clock_samples) int64 18MB ...
    signal_data_stroke_rate       (stroke_rate_samples) float64 18MB ...
    signal_data_light             (light_samples) float64 18MB ...
    signal_data_velocity          (velocity_samples) float64 18MB ...
    signal_data_prh               (prh_samples, prh_variables) float64 72MB ...
    ...                            ...
    event_data_type               (event_data_samples) <U5 40B ...
    event_data_key                (event_data_samples) <U13 104B ...
    event_data_value              (event_data_samples) float64 16B ...
    event_data_duration           (event_data_samples) float64 16B ...
    event_data_short_description  (event_data_samples) <U18 144B ...
    event_data_long_description   (event_data_samples) <U51 408B ...
Attributes: (12/26)
    deployment_info_Deployment Date:          2015-11-06
    deployment_info_Deployment Latitude:      -42.57286103836319
    deployment_info_Deployment Longitude:     -63.58798419615338
    deployment_info_Time Zone:                America/Catamarca
    animal_info_Animal_ID:                    mile-014
    dataset_info_Dataset_ID:                  mile-adult-sese_vdr_argentina_R...
    ...                                       ...
    signal_info_position_channels_3:          geopos_y_perdive
    signal_info_logger_status_channels_0:     wet_dry
    signal_info_feeding_attempts_channels_0:  feeding
    signal_info_location_channels_0:          latitude
    signal_info_location_channels_1:          longitude
    signal_info_depth_channels_0:             corrected_depth

In [2]:
df_event_types

,key,short_description,long_description,type,duration,group,processing_step
0,stroke_detected,Swimming stroke detected,Automatically detected swimming stroke from ac...,point,0.0,behavioral,Step 04: Stroke Detection & ODBA
1,heartbeat_auto_detect_accepted,auto-detected heartbeat (accepted),Automatically detected heartbeat that passed q...,point,0.0,physiological,Step 05: Heartbeat Detection
2,heartbeat_auto_detect_rejected,auto-detected heartbeat (rejected as spurious ...,Automatically detected heartbeat that was reje...,point,0.0,physiological,Step 05: Heartbeat Detection
3,heartbeat_auto_detect_suggested,"auto-detected heartbeat (suggested, missed-bea...",Automatically suggested heartbeat to fill gaps...,point,0.0,physiological,Step 05: Heartbeat Detection
4,heartbeat_auto_detect_gap,interval where HR was invalid (>50% jump / too...,Time interval where heart rate was deemed inva...,state,variable,physiological,Step 05: Heartbeat Detection
5,user_custom,Custom user annotation,User-defined annotation created via Dash inter...,point or state,"0 for point, variable for state",user_annotations,User Interface
6,dive_start,Start of dive,Manually annotated start of diving behavior,point,0.0,behavioral,User Interface
7,dive_end,End of dive,Manually annotated end of diving behavior,point,0.0,behavioral,User Interface
8,feeding_bout,Feeding behavior,Observed or inferred feeding behavior,state,variable,behavioral,User Interface
9,behavior_annotation,General behavior annotation,User-defined behavioral annotation for any obs...,state,variable,behavioral,User Interface
